<a href="https://colab.research.google.com/github/dannys0n/CS394-MyModulesRepo/blob/main/notebooks/04/Image%20or%20text%20to%20Unity%20PBR%20Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Module 4

<a target="_blank" href="https://colab.research.google.com/github/dannys0n/CS394-MyModulesRepo/blob/main/notebooks/04/Image%20or%20text%20to%20Unity%20PBR%20Pipeline.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://github.com/dannys0n/CS394-MyModulesRepo/blob/main/notebooks/04/Image%20or%20text%20to%20Unity%20PBR%20Pipeline.ipynb">
  <img src="https://img.shields.io/badge/Download_.ipynb-blue" alt="Download .ipynb"/>
</a>

Replicate Model Pipeline

In [ ]:

!uv pip install -q replicate gradio==5.49.1


In [ ]:

import os
import tempfile
import numpy as np
import replicate
import gradio as gr
from PIL import Image
import sys
import io
from dotenv import load_dotenv
from PIL import Image
import zipfile
from typing import Optional
from replicate.helpers import FileOutput
import requests


## Set Replicate API Token

In [ ]:

if 'google.colab' in sys.modules:
  from google.colab import userdata # type:ignore
  REPLICATE_API_TOKEN = userdata.get('REPLICATE_API_TOKEN')
else:
  load_dotenv()
  REPLICATE_API_TOKEN = os.getenv('REPLICATE_API_TOKEN')


client = replicate.Client(api_token=REPLICATE_API_TOKEN)

## Utility Helpers

In [ ]:

def get_albedo(
    prompt: str,
    uploaded_albedo: Optional[Image.Image],
    client
) -> Image.Image:
    # Returns an albedo image either by:
    # - using the uploaded image, or
    # - generating one from a text prompt via Replicate

    if uploaded_albedo is not None:
        return uploaded_albedo

    if not prompt or prompt.strip() == "":
        raise ValueError("Either a prompt or an albedo image is required.")

    try:
      output = client.run(
          "tstramer/material-diffusion:a42692c54c0f407f803a0a8a9066160976baedb77c91171a01730f9b0d7beeff",
          input={
              "prompt": prompt
          }
      )
    except Exception as e:
        raise RuntimeError(f"Albedo generation failed: {e}")

    return replicate_image_to_pil(output)

def pil_to_bytes(image):
    # Converts a PIL Image into a PNG byte stream for Replicate.
    buffer = io.BytesIO()
    image.save(buffer, format="PNG")
    buffer.seek(0)
    return buffer


def replicate_image_to_pil(output) -> Image.Image:
    # Converts Replicate outputs (bytes, URL, FileOutput) into a PIL Image.

    # Case 1: FileOutput (most common)
    if isinstance(output, FileOutput):
        response = requests.get(output.url)
        response.raise_for_status()
        return Image.open(io.BytesIO(response.content))

    # Case 2: URL string
    if isinstance(output, str):
        response = requests.get(output)
        response.raise_for_status()
        return Image.open(io.BytesIO(response.content))

    # Case 3: raw bytes
    if isinstance(output, (bytes, bytearray)):
        return Image.open(io.BytesIO(output))

    # Case 4: list outputs (some models)
    if isinstance(output, list):
        return replicate_image_to_pil(output[0])

    raise TypeError(f"Unsupported Replicate output type: {type(output)}")


## Pipeline Steps

In [ ]:
def generate_depth(albedo: Image.Image, client) -> Image.Image:
    output = client.run(
        "chenxwh/depth-anything-v2:b239ea33cff32bb7abb5db39ffe9a09c14cbc2894331d1ef66fe096eed88ebd4",
        input={"image": pil_to_bytes(albedo), "model_size": "Large"}
    )
    return Image.open(io.BytesIO(output["grey_depth"].read()))


def generate_normals(albedo: Image.Image, client) -> Image.Image:
    output = client.run(
        "tommoore515/pix2pix_tf_albedo2pbrmaps:21bd96b6e69f40e54502d67798f9025ab9e4a9e08f2a1b51dde5131b129a825e",
        input={"model": "albedo2normal", "imagepath": pil_to_bytes(albedo)}
    )
    return replicate_image_to_pil(output)


def generate_ao(albedo: Image.Image, client) -> Image.Image:
    output = client.run(
        "tommoore515/pix2pix_tf_albedo2pbrmaps:21bd96b6e69f40e54502d67798f9025ab9e4a9e08f2a1b51dde5131b129a825e",
        input={"model": "height2ao", "imagepath": pil_to_bytes(albedo)}
    )
    return replicate_image_to_pil(output)

def save_texture(image: Image.Image, path: str):
    image.save(path, format="PNG")


def export_unity_pbr(
    albedo: Image.Image,
    normal: Image.Image,
    depth: Image.Image,
    ao: Image.Image
) -> str:
    """
    Packages textures into a Unity-ready ZIP.
    Returns the path to the zip file.
    """

    export_dir = "unity_export/PBR_Material"
    os.makedirs(export_dir, exist_ok=True)

    # --- SAVE FILES ---
    save_texture(albedo, os.path.join(export_dir, "Albedo.png"))
    save_texture(normal, os.path.join(export_dir, "Normal.png"))
    save_texture(depth,  os.path.join(export_dir, "Height.png"))
    save_texture(ao,     os.path.join(export_dir, "AO.png"))

    zip_path = "PBR_Material_Unity.zip"

    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
        for file in os.listdir(export_dir):
            zipf.write(
                os.path.join(export_dir, file),
                arcname=file
            )

    return zip_path


## Gradio UI

In [ ]:
def run_pipeline(prompt, uploaded_albedo):
    # Streams results to Gradio as they are generated.

    # placeholders (None keeps old values)
    depth = normals = albedo = ao = zip_path = None

    # --- ALBEDO ---
    albedo = get_albedo(prompt, uploaded_albedo, client)
    yield albedo, depth, normals, ao, zip_path

    # --- DEPTH ---
    depth = generate_depth(albedo, client)
    yield albedo, depth, normals, ao, zip_path

    # --- NORMALS ---
    normals = generate_normals(depth, client)
    yield albedo, depth, normals, ao, zip_path

    # --- AO ---
    ao = generate_ao(depth, client)
    yield albedo, depth, normals, ao, zip_path

    # --- ZIP ---
    zip_path = export_unity_pbr(albedo, normals, depth, ao)
    yield albedo, depth, normals, ao, zip_path

gr.Interface(
    fn=run_pipeline,
    inputs=[
        gr.Textbox(label="Generate Albedo"),
        gr.Image(label="Upload Albedo", type="pil")
    ],
    outputs=[
        gr.Image(label="Albedo"),
        gr.Image(label="Depth Map"),
        gr.Image(label="Normal Map"),
        gr.Image(label="Ambient Occlusion"),
        gr.File(label="Unity Export (.zip)")
    ],
    title="Image → Unity PBR Material Pipeline"
).launch()

###Which option you chose and why:
Option 2 - I wanted to do something that would generate practical value for rapid prototying.

###Your design decisions and approach:
Some PBR generating models already exist, I opted to mix and match for academic learning. Importing a base image is always more reliable, but I wanted the flexibility to generating one as well.

###Observations about model behavior, quality, or limitations:
Generating an Albedo map often yielded unrealiable results, the biggest being baked-in shadows and occlusion, despite best efforts in prompting. Metalic PBR textures seemed to generally come out nicer when importing to Unity.
As much as I wanted to make this drag and drop, I could not be bothered to scour the web for a metallic map generator

###What worked well and what was challenging:
Graphics are not my specialty, so there was likely a mistake or two from generation or importing to Unity.
The biggest issue was models taking its time warming up. This could probably be mitigated my prewarming and parallel prompting.

###Some gifs of importing generated PBRs into Unity using URP/Lit shader

In [ ]:
from IPython.display import Image, display
import requests

# base image generation derived from here: https://raw.githubusercontent.com/simonguest/CS-394/refs/heads/main/src/04/images/dragon.png
gif_bytes = requests.get("https://raw.githubusercontent.com/dannys0n/CS394-MyModulesRepo/refs/heads/main/notebooks/04/pbr_dragon.gif").content
display(Image(data=gif_bytes))

# this one was generating an albedo from prompt
gif_bytes = requests.get("https://raw.githubusercontent.com/dannys0n/CS394-MyModulesRepo/refs/heads/main/notebooks/04/pbr_wall.gif").content
display(Image(data=gif_bytes))

## Use of AI Tools (Academic Disclosure)

AI-assisted tools (ChatGPT-5.2 Auto) were used during the development of this notebook to:
- Assist in refining Python code
- Assist in API specific syntax
- Debug runtime and UI issues

All generated content was reviewed, modified, and validated by the author.
The author retains responsibility for the correctness and originality of the final submission.